In [8]:
import json
from influxdb import InfluxDBClient
from time import time
import websocket

In [9]:
env_file = '.env'
with open(env_file) as f:
    env_data = json.load(f)
    
HUB_ADDR = env_data["hubitat_ip"]
INFLUX_ADDR = env_data["influx_ip"]
USER = env_data["username"]
PASS = env_data["password"]

e_sock = f"ws://{HUB_ADDR}/eventsocket"

In [10]:
def message_parser(message):
    """
    Exract info required for data insertion
    """
    
    # Used to remap values if needed
    value_mapping = {
        "on": 1,
        "off": 0
    }
    
    measurement=message["name"]
    displayName=message["displayName"]
    value=message["value"]
    unit=message["unit"]
    did=message["deviceId"]
    timestamp=message["ts"]
    if value in value_mapping.keys():
        value = value_mapping[value]

    try:
        value=float(value)
    except ValueError as e:
        # YES I'm passing this - this is because the incoming value is actually a string, let it free
        pass
    
    if type(value) == str:
        line = f'{measurement},deviceId={did} value="{value}",unit="{unit}",displayName="{displayName}" {int(timestamp*1e9)}'
    else:
        line = f'{measurement},deviceId={did} value={value},unit="{unit}",displayName="{displayName}" {int(timestamp*1e9)}'
    print(line)
    return line

In [11]:
def insert_message(wsapp, message):
    """
    Write the message to the database after parsing
    """
    message = json.loads(message)
    message["ts"] = time()
    line = message_parser(message)
    client =InfluxDBClient(
        host=INFLUX_ADDR,
        port=8086,
        username=USER,
        password=PASS)

    client.write([line], {'db': 'hubitat'}, 204, 'line')
    client.close()

In [12]:
wsapp = websocket.WebSocketApp(e_sock, on_message=insert_message)
wsapp.run_forever()

battery,deviceId=2 value=6.0,unit="%",displayName="Living room temperature" 1641161858496754176
temperature,deviceId=2 value=65.07,unit="°F",displayName="Living room temperature" 1641161858513209600
humidity,deviceId=2 value=32.0,unit="%",displayName="Living room temperature" 1641161858607134720
pressure,deviceId=2 value=988.5,unit="mbar",displayName="Living room temperature" 1641161858703175680
temperature,deviceId=1 value=67.0,unit="°F",displayName="Home thermostat" 1641161914713707776
temperature,deviceId=1 value=67.0,unit="°F",displayName="Home thermostat" 1641161914758830592
battery,deviceId=43 value=2.0,unit="%",displayName="Basement temp" 1641162508113442816
temperature,deviceId=43 value=60.28,unit="°F",displayName="Basement temp" 1641162508125943552
pressure,deviceId=43 value=989.1,unit="mbar",displayName="Basement temp" 1641162508193814784
humidity,deviceId=43 value=30.0,unit="%",displayName="Basement temp" 1641162508729617664
temperature,deviceId=43 value=60.57,unit="°F",disp

False